In [1]:
import torch
import segmentation_models_pytorch as smp
import datasets
import datetime
import numpy as np

from myUtils import *

In [2]:
res = 320
dataset = datasets.PlateDataset(res = res, train = True)

In [3]:
#plot_tensor(dataset[0])

In [4]:
ENCODER = 'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
DEVICE = 'cuda'

# create segmentation model with pretrained encoder
model = smp.FPN(
    encoder_name=ENCODER,
    encoder_weights=ENCODER_WEIGHTS,
    classes=3,
    activation='sigmoid',
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [5]:
# Dice/F1 score - https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient
# IoU/Jaccard score - https://en.wikipedia.org/wiki/Jaccard_index

loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

In [6]:
# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    model,
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

In [7]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True, num_workers=12)

In [8]:
# append to file
score = 0
optimizer.param_groups[0]['lr'] = 1e-4
for i in range(0, 150):
    print(f'\nEpoch: {i}')
    train_logs = train_epoch.run(dataloader)
    
    if train_logs['iou_score'] > score:
        torch.save(model, f'./best_model.pth')
    else:
        torch.save(model, f'./checkpoint_model.pth')

    score = train_logs['iou_score']

    with open('./train.log', 'a') as f:
        f.write(f'{i} | {datetime.datetime.now()} | {str(train_logs)}\n')

    if i == 50:
        optimizer.param_groups[0]['lr'] = 1e-5
        print('Decrease decoder learning rate to 1e-5!')

    if i == 75:
        optimizer.param_groups[0]['lr'] = 1e-6
        print('Decrease decoder learning rate to 1e-6!')

    if i == 100:
        optimizer.param_groups[0]['lr'] = 1e-7
        print('Decrease decoder learning rate to 1e-7!')


Epoch: 0
train:   0%|          | 0/56 [00:00<?, ?it/s]

/root/workspace/tracking/venv/lib/python3.9/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


train: 100%|██████████| 56/56 [00:22<00:00,  2.50it/s, dice_loss - 0.386, iou_score - 0.4681] 

Epoch: 1
train: 100%|██████████| 56/56 [00:22<00:00,  2.50it/s, dice_loss - 0.2366, iou_score - 0.6261]

Epoch: 2
train: 100%|██████████| 56/56 [00:22<00:00,  2.52it/s, dice_loss - 0.2117, iou_score - 0.6589]

Epoch: 3
train: 100%|██████████| 56/56 [00:22<00:00,  2.49it/s, dice_loss - 0.1941, iou_score - 0.6825]

Epoch: 4
train: 100%|██████████| 56/56 [00:21<00:00,  2.59it/s, dice_loss - 0.1824, iou_score - 0.6991]

Epoch: 5
train: 100%|██████████| 56/56 [00:19<00:00,  2.82it/s, dice_loss - 0.1689, iou_score - 0.7189]

Epoch: 6
train: 100%|██████████| 56/56 [00:19<00:00,  2.80it/s, dice_loss - 0.1677, iou_score - 0.7206]

Epoch: 7
train: 100%|██████████| 56/56 [00:20<00:00,  2.79it/s, dice_loss - 0.1532, iou_score - 0.7419]

Epoch: 8
train: 100%|██████████| 56/56 [00:20<00:00,  2.79it/s, dice_loss - 0.1508, iou_score - 0.7442]

Epoch: 9
train: 100%|██████████| 56/56 [00:20<00:00,  2.78it/s, 

In [9]:
model = torch.load(f'./best_model.pth')

In [10]:
for i in range(10):
    rand = np.random.randint(0, len(dataset))
    plot_pred(dataset[rand][0], model)

RuntimeError: shape '[2, 320, 320]' is invalid for input of size 307200